# Data Preparation

In [1]:
# Importieren der notwendigen Bibliotheken
import pandas as pd
import holidays
import os

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



In [2]:
# Laden der Daten
# Passe den Pfad entsprechend an, falls die Daten in einem spezifischen Ordner liegen
df = pd.read_csv('../data/raw/sickness_table.csv', parse_dates=['date'])

In [3]:
# Unnötige Index-Spalte ignorieren
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df

,date,n_sick,calls,n_duty,n_sby,sby_need,dafted
0,2016-04-01,73,8154.0,1700,90,4.0,0.0
1,2016-04-02,64,8526.0,1700,90,70.0,0.0
2,2016-04-03,68,8088.0,1700,90,0.0,0.0
3,2016-04-04,71,7044.0,1700,90,0.0,0.0
4,2016-04-05,63,7236.0,1700,90,0.0,0.0
...,...,...,...,...,...,...,...
1147,2019-05-23,86,8544.0,1900,90,0.0,0.0
1148,2019-05-24,81,8814.0,1900,90,0.0,0.0
1149,2019-05-25,76,9846.0,1900,90,146.0,56.0
1150,2019-05-26,83,9882.0,1900,90,160.0,70.0


In [5]:
# Übersetzen der 

## 1. Datenbereinigung (Teil 1)

In der Datenanalyse wurden folgende Aspekte der Datenqualität überprüft:

- **Fehlende Werte:** Es wurde festgestellt, dass keine fehlenden Werte in den Daten vorliegen. Somit sind keine Imputationsmethoden oder Auffüllstrategien notwendig.
- **Duplikate:** Die Daten wurden auf Duplikate überprüft, und es konnte bestätigt werden, dass keine doppelten Einträge vorhanden sind. Ein Entfernen doppelter Zeilen ist daher nicht erforderlich.
- **Ausreißer:** Während der Analyse wurden mögliche Ausreißer in den numerischen Variablen untersucht. Die vorhandenen Ausreißer scheinen jedoch alle plausibel und nachvollziehbar zu sein (z. B. Spitzenwerte bei Notrufen oder Krankenständen zu bestimmten Zeiten). Daher wurde entschieden, die Ausreißer nicht weiter zu beschneiden oder zu entfernen.

In [6]:
# Float-Spalten in int64 umwandeln (calls, sby_need und dafted)
df['calls'] = df['calls'].astype('int64')
df['sby_need'] = df['sby_need'].astype('int64')
df['dafted'] = df['dafted'].astype('int64')

In [7]:
# Spaltennamen ins Deutsche übersetzen
df = df.rename(columns={
    'date': 'Datum',
    'n_sick': 'Anzahl_Krankenstand',
    'calls': 'Anzahl_Notrufe',
    'n_duty': 'Anzahl_im_Dienst',
    'n_sby': 'Anzahl_Ersatzfahrer_Bereitschaft',
    'sby_need': 'Ersatzfahrer_aktiviert',
    'dafted': 'Zusätzliche_Fahrer_erforderlich'
})

## 2. Feature Engineering

In [8]:
# Monat, Wochentag, Quartal und Jahreszeit aus dem Datum extrahieren
df['Monat'] = df['Datum'].dt.month
df['Wochentag'] = df['Datum'].dt.dayofweek  # 0=Montag, 6=Sonntag
df['Quartal'] = df['Datum'].dt.quarter
df['Jahreszeit'] = df['Monat'] % 12 // 3 + 1  # 1=Winter, 2=Frühling, 3=Sommer, 4=Herbst


In [9]:
# Lag-Features für 7 und 30 Tage für relevante Spalten erstellen
for lag in [7, 30]:
    df[f'Anzahl_Notrufe_Lag_{lag}'] = df['Anzahl_Notrufe'].shift(lag)
    df[f'Anzahl_Krankenstand_Lag_{lag}'] = df['Anzahl_Krankenstand'].shift(lag)
    df[f'Ersatzfahrer_aktiviert_Lag_{lag}'] = df['Ersatzfahrer_aktiviert'].shift(lag)
    df[f'Zusätzliche_Fahrer_erforderlich_Lag_{lag}'] = df['Zusätzliche_Fahrer_erforderlich'].shift(lag)


In [10]:
# Berechnung von 7-Tage und 30-Tage gleitenden Durchschnitten und Varianzen
for window in [7, 30]:
    df[f'Anzahl_Notrufe_MA_{window}'] = df['Anzahl_Notrufe'].rolling(window=window).mean()
    df[f'Anzahl_Krankenstand_MA_{window}'] = df['Anzahl_Krankenstand'].rolling(window=window).mean()
    df[f'Ersatzfahrer_aktiviert_MA_{window}'] = df['Ersatzfahrer_aktiviert'].rolling(window=window).mean()
    df[f'Zusätzliche_Fahrer_erforderlich_MA_{window}'] = df['Zusätzliche_Fahrer_erforderlich'].rolling(window=window).mean()

    df[f'Anzahl_Notrufe_Var_{window}'] = df['Anzahl_Notrufe'].rolling(window=window).var()
    df[f'Anzahl_Krankenstand_Var_{window}'] = df['Anzahl_Krankenstand'].rolling(window=window).var()
    df[f'Ersatzfahrer_aktiviert_Var_{window}'] = df['Ersatzfahrer_aktiviert'].rolling(window=window).var()
    df[f'Zusätzliche_Fahrer_erforderlich_Var_{window}'] = df['Zusätzliche_Fahrer_erforderlich'].rolling(window=window).var()


In [11]:
# Entfernen von Zeilen mit NaN-Werten (bedingt durch Lag- und Rolling-Features)
df = df.dropna().reset_index(drop=True)

Durch die oben genannten Schritte wurde die Zeitstruktur der Daten aufgeschlüsselt und wichtige historische Informationen eingebracht. Diese vorbereiteten Features bieten eine starke Grundlage für das Modelltraining, indem sie saisonale Muster und historische Trends berücksichtigen. Mit diesen zusätzlichen Informationen kann das Modell saisonale Zyklen und vergangenheitsbasierte Abhängigkeiten effektiver erfassen, was die Genauigkeit der Vorhersagen verbessern sollte.

## 1. Zeitbasierte Features extrahieren
- Erstellte Features:
    - **Monat:** Gibt den Monat des jeweiligen Datums an (Werte: 1 bis 12).
    - **Wochentag:** Gibt den Wochentag des jeweiligen Datums an (Werte: 0 für Montag bis 6 für Sonntag).
    - **Quartal:** Zeigt an, in welchem Quartal das Datum liegt (Werte: 1 bis 4).
    - **Jahreszeit:** Zeigt die Jahreszeit an, in der das Datum liegt (Werte: 1 für Winter, 2 für Frühling, 3 für Sommer, 4 für Herbst).
- **Begründung:** Zeitbasierte Features sind entscheidend für Zeitreihen, da sie saisonale Muster und Zyklen erfassen. Zum Beispiel können Notrufzahlen und Krankenstände zu bestimmten Jahreszeiten oder Wochentagen höher oder niedriger sein. Durch das Hinzufügen von Features wie Monat, Wochentag, Quartal und Jahreszeit kann das Modell saisonale Schwankungen und wöchentliche Trends besser erkennen.

## 2. Lag-Features erstellen
- Erstellte Features:
    - **Lag-Features:** Werte der Variablen Anzahl_Notrufe, Anzahl_Krankenstand, Ersatzfahrer_aktiviert und Zusätzliche_Fahrer_erforderlich mit Verzögerungen von 7 und 30 Tagen (z. B. Anzahl_Notrufe_Lag_7 und Anzahl_Notrufe_Lag_30).
- **Begründung:** Lag-Features ermöglichen es dem Modell, auf vergangene Informationen zuzugreifen und Trends oder Muster in der Vergangenheit zu berücksichtigen. Für Zeitreihen ist es besonders nützlich zu wissen, wie sich eine Variable in den vorhergehenden Tagen entwickelt hat. Zum Beispiel könnte die Anzahl der Notrufe vor 7 oder 30 Tagen einen Einfluss auf die heutige Anzahl haben. Durch die Einführung von 7-Tage- und 30-Tage-Lags erhält das Modell Informationen über wöchentliche und monatliche Verzögerungen, was bei der Vorhersage saisonaler Muster hilft.

## 3. Gleitende Durchschnitte und Varianzen berechnen
- Erstellte Features:
    - **Gleitende Durchschnitte:** 7-Tage- und 30-Tage-Durchschnitte der Variablen Anzahl_Notrufe, Anzahl_Krankenstand, Ersatzfahrer_aktiviert und Zusätzliche_Fahrer_erforderlich (z. B. Anzahl_Notrufe_MA_7 und Anzahl_Notrufe_MA_30).
    - **Gleitende Varianzen:** 7-Tage- und 30-Tage-Varianzen der gleichen Variablen (z. B. Anzahl_Notrufe_Var_7 und Anzahl_Notrufe_Var_30).
- **Begründung:** Gleitende Durchschnitte (Moving Averages) glätten kurzfristige Schwankungen und geben dem Modell ein klareres Bild von längerfristigen Trends. Die 7-Tage-Durchschnitte erfassen wöchentliche Muster, während die 30-Tage-Durchschnitte längere saisonale Trends widerspiegeln. Gleitende Varianzen (Moving Variances) geben an, wie stark die Werte innerhalb eines bestimmten Zeitfensters schwanken, was Hinweise auf die Stabilität oder Instabilität eines Merkmals geben kann. Wenn beispielsweise die Varianz in den Krankenständen in bestimmten Zeitfenstern hoch ist, könnte dies auf besondere Ereignisse oder Einflüsse hindeuten.

## 4. Entfernen von NaN-Werten
- Durchgeführte Schritte:
    - Entfernen von Zeilen mit NaN-Werten, die durch die Erstellung der Lag- und Moving Average-Features entstanden sind.
- **Begründung:** Lag- und Rolling-Funktionen erzeugen am Anfang der Zeitreihe NaN-Werte, da für die ersten Zeilen keine Daten für die Berechnung der zurückliegenden Tage vorhanden sind. Diese Zeilen wurden entfernt, da sie keine vollständigen Daten enthalten und daher für das Modelltraining ungeeignet wären.


In [12]:
# Deutsche Feiertage für relevante Jahre erstellen
de_holidays = holidays.Germany(years=range(df['Datum'].dt.year.min(), df['Datum'].dt.year.max() + 1))


In [13]:
# Neues Feature hinzufügen: Feiertag (1 = Feiertag, 0 = kein Feiertag)
df['Feiertag'] = df['Datum'].apply(lambda x: 1 if x in de_holidays else 0)

In [14]:
# Wochenende hinzufügen (1 = Wochenende, 0 = kein Wochenende)
df['Wochenende'] = df['Datum'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

## 5. Feiertag
- **Beschreibung:** In der Spalte Feiertag wird überprüft, ob das jeweilige Datum in die Liste der deutschen Feiertage fällt. Dies wurde mithilfe der holidays-Bibliothek umgesetzt, die für Deutschland und spezifische Jahre Feiertage ausgibt. Für Feiertage erhält das Feature den Wert 1, für normale Tage den Wert 0.
 
- **Begründung:** Feiertage können einen wesentlichen Einfluss auf das Verhalten und die Nachfrage im Rettungsdienst haben. An Feiertagen kann es z. B. zu einer erhöhten Anzahl von Notrufen kommen (z. B. an Silvester) oder die Krankenstände können abweichen. Diese Tage haben oft einzigartige Muster, die ein Modell ohne Berücksichtigung der Feiertage nicht erkennen könnte. Durch das Hinzufügen dieses Features erhält das Modell zusätzliche Informationen, die helfen, besondere Tage zu identifizieren und diese in der Prognose zu berücksichtigen.
## 6. Wochenende
- **Beschreibung:** Die Spalte Wochenende gibt an, ob das Datum auf ein Wochenende fällt (Samstag oder Sonntag). Dabei erhalten Samstage und Sonntage den Wert 1, alle anderen Tage den Wert 0.

- **Begründung:** Wochenenden haben häufig andere Muster im Vergleich zu Werktagen, sowohl in Bezug auf Notrufe als auch auf Krankenstände. Beispielsweise kann die Anzahl der Notrufe am Wochenende variieren, da die Aktivität in der Bevölkerung anders ist als an Werktagen. Auch die Krankenstände könnten am Wochenende anders verlaufen. Dieses Feature hilft dem Modell, die Unterschiede zwischen Wochentagen und Wochenenden zu lernen und die Vorhersage entsprechend zu verbessern.

## Anpassen der Datentypen

In [15]:
# Kategorische Variablen als "category" festlegen
df['Monat'] = df['Monat'].astype('category')
df['Wochentag'] = df['Wochentag'].astype('category')
df['Quartal'] = df['Quartal'].astype('category')
df['Jahreszeit'] = df['Jahreszeit'].astype('category')

# Binäre Variablen als "bool" festlegen
df['Feiertag'] = df['Feiertag'].astype(bool)
df['Wochenende'] = df['Wochenende'].astype(bool)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Datum                                   1122 non-null   datetime64[ns]
 1   Anzahl_Krankenstand                     1122 non-null   int64         
 2   Anzahl_Notrufe                          1122 non-null   int64         
 3   Anzahl_im_Dienst                        1122 non-null   int64         
 4   Anzahl_Ersatzfahrer_Bereitschaft        1122 non-null   int64         
 5   Ersatzfahrer_aktiviert                  1122 non-null   int64         
 6   Zusätzliche_Fahrer_erforderlich         1122 non-null   int64         
 7   Monat                                   1122 non-null   category      
 8   Wochentag                               1122 non-null   category      
 9   Quartal                                 1122 non-nul

- **Kategorische Variablen (category):** Die Variablen Monat, Wochentag, Quartal und Jahreszeit sind als category definiert, was für spätere One-Hot-Encodings und ähnliche Transformationen nützlich ist.
- **Binäre Variablen (bool):** Die Variablen Feiertag und Wochenende sind als bool definiert, da sie nur zwei mögliche Werte haben (True/False) und als logische Indikatoren dienen.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Datum                                   1122 non-null   datetime64[ns]
 1   Anzahl_Krankenstand                     1122 non-null   int64         
 2   Anzahl_Notrufe                          1122 non-null   int64         
 3   Anzahl_im_Dienst                        1122 non-null   int64         
 4   Anzahl_Ersatzfahrer_Bereitschaft        1122 non-null   int64         
 5   Ersatzfahrer_aktiviert                  1122 non-null   int64         
 6   Zusätzliche_Fahrer_erforderlich         1122 non-null   int64         
 7   Monat                                   1122 non-null   category      
 8   Wochentag                               1122 non-null   category      
 9   Quartal                                 1122 non-nul

# Train-Test-Split

In [18]:
# Setze die Zielvariable
target = 'Ersatzfahrer_aktiviert'

# Definiere die Merkmale (X) und die Zielvariable (y)
X = df.drop(columns=[target])
y = df[target]

# Führe einen zeitlichen Train-Test-Split durch (80 % Training, 20 % Test) mit shuffle=False
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Überprüfen der Aufteilung
print("Training Set Größe:", X_train.shape, y_train.shape)
print("Test Set Größe:", X_test.shape, y_test.shape)

Training Set Größe: (897, 36) (897,)
Test Set Größe: (225, 36) (225,)


Nach dem Data Preprocessing (Bereinigung und Enrichment) habe ich die Daten in Trainings- und Testsets aufgeteilt. Diese Aufteilung dient dazu, das Modell auf historischen Daten zu trainieren und seine Leistung auf zukünftigen, bislang unbekannten Daten zu bewerten. Dies ist besonders wichtig für die Zeitreihenprognose, da wir sicherstellen möchten, dass das Modell keine „Zukunftsinformationen“ aus dem Testset erhält.

# Begründung
- **Vermeidung von Datenlecks:** Da es sich um zeitbasierte Daten handelt, ist es entscheidend, dass die Testdaten den Modellaufbau nicht beeinflussen, um die zukünftige Vorhersageleistung korrekt abzubilden. Ein zeitlicher Split verhindert, dass Informationen aus der „Zukunft“ in das Modelltraining einfließen.
- **Replizierbarkeit und Robustheit:** Die feste Datenaufteilung gewährleistet, dass die Modellleistung auch bei zukünftigen Daten realistisch ist und nicht auf Muster im Testset abgestimmt wurde.
- **Verallgemeinerungsfähigkeit:** Dieser Ansatz stellt sicher, dass das Modell generalisiert und saisonale Trends sowie kurzfristige Schwankungen auf Basis der Trainingsdaten erkennt.

# Encoding, Skalierung und PCA bzw. Feature Selection

In [30]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Merkmalsauswahl über Datentypen
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# 1. ColumnTransformer für Lineare Regression und Neuronale Netze mit PCA
preprocessor_lr_pca = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)
        ])),
    ('pca', PCA(n_components=0.95))  # Behalte 95% der Varianz
])

# 2. ColumnTransformer für Random Forest und Gradient Boosting ohne Feature Selection
preprocessor_tree_fs = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)  # Alternativ: Label-Encoding
        ]))
])

# 3. Fit und Transform für Trainingsdaten
X_train_lr_pca = preprocessor_lr_pca.fit_transform(X_train, y_train)
X_train_tree_fs = preprocessor_tree_fs.fit_transform(X_train, y_train)

# 4. Transform für Testdaten
X_test_lr_pca = preprocessor_lr_pca.transform(X_test)
X_test_tree_fs = preprocessor_tree_fs.transform(X_test)

## Modellabhängige Datenvorbereitung
Ich habe mich entschieden, unterschiedliche Pipelines für verschiedene Modellgruppen zu definieren, um eine effiziente und modellgerechte Datenvorbereitung zu ermöglichen. Zusätzlich habe ich PCA und Feature Selection integriert, um die Dimensionen zu reduzieren und die relevantesten Merkmale für jedes Modell hervorzuheben.

- Lineare Regression (Baseline-Modell)
    - **Zweck:** Die Lineare Regression dient als Baseline-Modell, um die Modellleistung mit einer einfachen, interpretierten Methode zu vergleichen.
    - **Encoding und Skalierung:** Für die Lineare Regression verwende ich One-Hot-Encoding für die kategorischen Variablen (Monat, Wochentag, Quartal, Jahreszeit), um den linearen Zusammenhang zwischen den einzelnen Kategorien und dem Zielwert explizit zu erfassen. Zudem wird ein StandardScaler auf die numerischen Variablen angewendet, da lineare Modelle empfindlich auf unterschiedliche Skalen reagieren.
    - **PCA:** Zusätzlich habe ich PCA (Principal Component Analysis) in die Pipeline integriert, um die Anzahl der Merkmale zu reduzieren und so die Berechnungen zu beschleunigen. Die PCA ist besonders hilfreich für die Lineare Regression, da sie das Modell auf die wichtigsten Hauptkomponenten fokussiert, ohne die interpretierbare Varianz zu verlieren. Die Anzahl der Hauptkomponenten wurde so gewählt, dass 95 % der Varianz erhalten bleiben, was eine gute Balance zwischen Komplexitätsreduktion und Informationserhalt bietet.

- Random Forest und Gradient Boosting
    - **Zweck:** Diese Modelle sind in der Lage, nichtlineare Zusammenhänge und Interaktionen zwischen den Variablen zu erfassen und robust gegen Ausreißer.
    - **Encoding und Skalierung:** Da baumbasierte Modelle unempfindlich gegenüber unterschiedlichen Skalen sind, lasse ich die numerischen Variablen unverändert (passthrough). Bei den kategorischen Variablen setze ich One-Hot-Encoding ein, um die Interpretierbarkeit der Features zu erhöhen und Redundanzen in den Baumstrukturen zu vermeiden. Alternativ könnte auch ein Label-Encoding für größere Datensätze verwendet werden, wenn Rechenzeit optimiert werden soll.


## Speicherung der prozessierten Daten

In [31]:
# Sicherstellen, dass der Ordner 'data/processed' existiert
os.makedirs('../data/processed', exist_ok=True)

# Speichern der Trainings- und Testdaten für lineare Modelle
pd.DataFrame(X_train_lr_pca).to_csv('../data/processed/X_train_lr_pca.csv', index=False)
pd.DataFrame(X_test_lr_pca).to_csv('../data/processed/X_test_lr_pca.csv', index=False)

# Speichern der Trainings- und Testdaten für baumbasierte Modelle
pd.DataFrame(X_train_tree_fs).to_csv('../data/processed/X_train_tree_fs.csv', index=False)
pd.DataFrame(X_test_tree_fs).to_csv('../data/processed/X_test_tree_fs.csv', index=False)

# Zielwerte speichern
pd.DataFrame(y_train).to_csv('../data/processed/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('../data/processed/y_test.csv', index=False)